# 2.10 Dual basis functions


### Canonical interpolation
The canonical finite element interpolation operator is defined by specifying the degrees of freedom. For low order methods these are typically nodal values, while for high order methods these are most often moments. For example, interpolating onto the order p triangle can be specified by: find $u_{hp} \in V_{hp}$ such that

\begin{eqnarray*}
u_{hp} (x) & = & u(x) \quad \forall \text{ vertices } V \\
\int_E u_{hp} q & = & \int_E u q \quad \forall q \in P^{p-2} \; \forall \text{ edges } E \\
\int_T u_{hp} q & = & \int_T u q \quad \forall q \in P^{p-3} \; \forall \text{ triangles } T
\end{eqnarray*}

Most NGSolve finite element spaces provide now a "dual" operator, which delivers now the moments (i.e. the dual space basis functions) instead of function values. The integrals over faces, edges and also vertices are defined by co-dimension 1 (=BND), co-dimension 2 (=BBND) or co-dimension 3 (=BBBND).

In [ ]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=2))

The typical NGSolve 'Set' function does local projection, and simple averaging. In particular, this does not provide point values in vertices. 

In [ ]:
fes = H1(mesh, order=3)  # , low_order_space=False)  for NGSolve1905

func = x*x*x*x
gfu = GridFunction(fes)
gfu.Set(func)
print (gfu.vec)

We define now a variational problem for canonical interpolaion. 

In [ ]:
u,v = fes.TnT()
vdual = v.Operator("dual")

a = BilinearForm(fes)
a += u*vdual*dx + u*vdual*dx(element_vb=BND) + \
    u*vdual*dx(element_vb=BBND)
a.Assemble()

f = LinearForm(fes)
f += func*vdual*dx + func*vdual*dx(element_vb=BND) + \
    func*vdual*dx(element_vb=BBND)
f.Assemble()

# interpolation in vertices preserves values 0 and 1
gfu.vec.data = a.mat.Inverse() * f.vec
print (gfu.vec)
Draw (gfu)

In [ ]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=2))

fesh1 = VectorH1(mesh, order=2)
feshdiv = HDiv(mesh, order=2)

gfuh1 = GridFunction(fesh1)
gfuh1.Set ( (x*x,y*y) )

amixed = BilinearForm(trialspace=fesh1, testspace=feshdiv)
ahdiv = BilinearForm(feshdiv)

u,v = feshdiv.TnT()
vdual = v.Operator("dual")
uh1 = fesh1.TrialFunction()

n = specialcf.normal(mesh.dim)
def tang(v) : return v - (v*n)*n

dS = dx(element_boundary=True)
ahdiv += u*vdual * dx + u*vdual * dS
ahdiv.Assemble()

amixed += uh1*vdual*dx + uh1*vdual*dS
amixed.Assemble()

print ("ahdiv =", ahdiv.mat)
print ("amixed =", amixed.mat)


gfuhdiv = GridFunction(feshdiv, name="ucomp")
transform = ahdiv.mat.Inverse() @ amixed.mat
gfuhdiv.vec.data = transform * gfuh1.vec

Draw (gfuh1)
Draw (gfuhdiv, mesh, "uhdiv")

# input ("key")

eblocks = []
for e in mesh.edges:
    eblocks.append ( feshdiv.GetDofNrs(e) )
fblocks = []
for f in mesh.faces:
    fblocks.append ( feshdiv.GetDofNrs(f) )

print (eblocks)
print (fblocks)

einv = ahdiv.mat.CreateBlockSmoother(eblocks)
finv = ahdiv.mat.CreateBlockSmoother(fblocks)

transform = (einv + finv @ (IdentityMatrix() - ahdiv.mat @ einv)) @ amixed.mat
gfuhdiv.vec.data = transform * gfuh1.vec

# input ("key")

temp = amixed.mat.CreateColVector()
temp.data = amixed.mat * gfuh1.vec
gfuhdiv.vec[:] = 0
einv.Smooth (gfuhdiv.vec, temp)
finv.Smooth (gfuhdiv.vec, temp)


## Auxiliary Space Preconditioning
use H1 preconditioner for what ? MCS ? 